In [5]:
import pandas as pd
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test_data.csv')
print("训练集探索")
print(df_train.head())
print(len(df_train))
print("测试集探索")
print(df_test.head())
print(len(df_test))

训练集探索
   id         1         2         3         4         5         6         7  \
0   1  0.563650  1.069229 -0.837759 -1.122021  0.433296  0.770755 -0.477153   
1   2  0.061333  0.058830  0.056952  0.068634  0.073433  0.072390  0.042975   
2   3  0.035736  0.010964 -0.164872 -0.167714 -0.125075 -0.104771 -0.016650   
3   4 -0.046700  0.060913  0.009340 -0.093400 -0.067817  0.022335  0.006091   
4   5  0.162922 -0.377662  0.014457  0.565437 -0.203369 -0.511508  0.410961   

          8         9  ...        5992      5993      5994      5995  \
0 -0.588421  0.455224  ...   -0.050761  0.220506  0.036548 -0.097461   
1 -0.007302 -0.026286  ...    0.061333  0.107437  0.104516  0.063419   
2  0.151471  0.137258  ...    4.272044 -1.991455 -2.922208  1.937039   
3 -0.076751 -0.032893  ...    0.095025 -0.000406  0.091776  0.074314   
4  0.228546 -0.515244  ...   -0.093563 -0.263632  0.114517  0.209541   

       5996      5997      5998      5999      6000  label  
0 -0.084060 -0.007716 -0.

In [3]:
# 测试DTW计算结果
import numpy as np
from DTW import DTW 
s1 = np.array([1, 2, 3, 4, 5, 5, 5, 4])
s2 = np.array([3, 4, 5, 5, 5, 4])
print(DTW(s1,s2))

3.0


In [6]:
# 提取训练集的y集
y_train = pd.DataFrame()
y_train['label'] = df_train['label']
df_train.drop(columns='label', inplace=True)

In [8]:
# 定义函数计算时序数据特征值，参考tsfresh，但是减少了特征量
import numpy as np
def get_t(df):
    mean_list = []
    std_list = []
    var_list = []
    min_list = []
    max_list = []
    median_list = []
    x = pd.DataFrame()
    for i in range(len(df)):
        mean_list.append(np.mean(df.iloc[i][1:].values))
        std_list.append(np.std(df.iloc[i][1:].values))
        var_list.append(np.var(df.iloc[i][1:].values))
        min_list.append(np.min(df.iloc[i][1:].values))
        max_list.append(np.max(df.iloc[i][1:].values))
        median_list.append(np.median(df.iloc[i][1:].values))
    x['mean'] = mean_list
    x['std'] = std_list
    x['var'] = var_list
    x['min'] = min_list
    x['max'] = max_list
    x['median'] = median_list
    return x

In [9]:
# 对训练集和测试集利用时序数据构造新特征
x_train = get_t(df_train)
x_test = get_t(df_test)
print(x_train.head())
print(x_test.head())

       mean       std       var       min       max    median
0  0.004765  0.583343  0.340289 -3.003019  3.236114  0.007919
1  0.015331  0.065739  0.004322 -0.222801  0.221132  0.017524
2  0.003270  0.540327  0.291954 -5.501272  5.912233  0.002843
3  0.004026  0.366301  0.134176 -3.955295  3.712455  0.003249
4  0.013258  0.233977  0.054745 -1.754299  1.550606  0.010802
       mean       std       var       min       max    median
0  0.003489  0.537096  0.288472 -4.631026  5.202391  0.003249
1  0.006061  0.809820  0.655808 -2.891751  3.168703 -0.003452
2  0.006581  0.361151  0.130430 -3.624334  3.813977  0.006497
3  0.009584  0.063453  0.004026 -0.243246  0.197559  0.011891
4  0.004482  0.152550  0.023271 -0.518818  0.556503  0.003411


In [11]:
# 使用sklearn中的KNN进行训练，尝试邻居数目优化不能对准确率有所改动
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(x_train, y_train.values.reshape(len(y_train), ))
# 进行结果预测
rst = knn.predict(x_test)
result = pd.DataFrame()
result['id'] = df_test['id']
result['label'] = rst
result.to_csv('data/result.csv', index=False, encoding="utf-8")

In [12]:
# 使用SVM进行预测，比对分数，发现表现很差，在增加时序提取的特征后表现较好，仍不比随机森林
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
clf = OneVsOneClassifier(LinearSVC(random_state=0))
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_train)
print(confusion_matrix(y_train, rst))

[[177   0   0   0   0   0   0   0   0   0]
 [  0  35   0  11   0   0   0   0   0   0]
 [  0   0  24  12   0   2   0   1   1   3]
 [  0   0   5  23   0   2   0   0  20   0]
 [  0   0   0   0  49   0   0   2   0   0]
 [  0   0   0   0   0  47   0   0   0   0]
 [  0   0   0   0   0   0  38   5   0   9]
 [  0   0   0   0   5   2   0 132   0   1]
 [  4   0   0   8   0   0   0   0  33   0]
 [  0   0   0   0   9   0   0   9   0 123]]


C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [13]:
# 随机森林在验证集上表现不错
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_train)
print(confusion_matrix(y_train, rst))

[[177   0   0   0   0   0   0   0   0   0]
 [  0  46   0   0   0   0   0   0   0   0]
 [  0   0  43   0   0   0   0   0   0   0]
 [  0   0   0  50   0   0   0   0   0   0]
 [  0   0   0   0  51   0   0   0   0   0]
 [  0   0   0   0   0  47   0   0   0   0]
 [  0   0   0   0   0   0  52   0   0   0]
 [  0   0   0   0   0   0   0 140   0   0]
 [  0   0   0   0   0   0   0   0  45   0]
 [  0   0   0   0   0   0   0   0   0 141]]


C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [14]:
# 随机森林在测试集上表现也不错，准确率达到94
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(x_train, y_train.values.reshape(len(y_train, )))
rst = clf.predict(x_test)
result = pd.DataFrame()
result['id'] = df_test['id']
result['label'] = rst
result.to_csv('data/result.csv', index=False, encoding="utf-8")

C:\Users\16957\Anaconda3\envs\data-analysis\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
